# k-Nearest Neighbors Classifier

In this notebook, you will implement your own k-nearest neighbors (k-NN) algorithm for the classification problem. You are supposed to learn:

* How to prepare the dataset for "training" and testing of the model.
* How to implement k-nearest neighbors classification algorithm.
* How to evaluate the performance of your classifier.

**Instructions:**

* Read carefuly through this notebook. Be sure you understand what is provided to you, and what is required from you.
* Place your code only in sections annotated with `### START CODE HERE ###` and `### END CODE HERE ###`.
* Use comments whenever the code is not self-explanatory.
* Submit an executable notebook (`*.ipynb`) with your solution to BlackBoard.

Enjoy :-)

## Packages

Following packages is all you need. Do not import any additional packages!

* [Pandas](https://pandas.pydata.org/) is a library providing easy-to-use data structures and data analysis tools.
* [Numpy](http://www.numpy.org/) library provides support for large multi-dimensional arrays and matrices, along with functions to operate on these.

In [14]:
import pandas as pd
import numpy as np

## Problem

You are given a dataset `mushrooms.csv` with characteristics/attributes of mushrooms, and your task is to implement and evaluate a k-nearest neighbors classifier able to say whether a mushroom is poisonous or edible based on its attributes.

## Dataset

The dataset of mushroom characteristics is freely available at [Kaggle Datasets](https://www.kaggle.com/uciml/mushroom-classification) where you can find further information about the dataset. It consists of 8124 mushrooms characterized by 23 attributes (including the class). Following is the overview of attributes and values:

* class: edible=e, poisonous=p
* cap-shape: bell=b,conical=c,convex=x,flat=f, knobbed=k,sunken=s
* cap-surface: fibrous=f,grooves=g,scaly=y,smooth=s
* cap-color: brown=n,buff=b,cinnamon=c,gray=g,green=r,pink=p,purple=u,red=e,white=w,yellow=y
* bruises: bruises=t,no=f
* odor: almond=a,anise=l,creosote=c,fishy=y,foul=f,musty=m,none=n,pungent=p,spicy=s
* gill-attachment: attached=a,descending=d,free=f,notched=n
* gill-spacing: close=c,crowded=w,distant=d
* gill-size: broad=b,narrow=n
* gill-color: black=k,brown=n,buff=b,chocolate=h,gray=g, green=r,orange=o,pink=p,purple=u,red=e,white=w,yellow=y
* stalk-shape: enlarging=e,tapering=t
* stalk-root: bulbous=b,club=c,cup=u,equal=e,rhizomorphs=z,rooted=r,missing=?
* stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s
* stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s
* stalk-color-above-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y
* stalk-color-below-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y
* veil-type: partial=p,universal=u
* veil-color: brown=n,orange=o,white=w,yellow=y
* ring-number: none=n,one=o,two=t
* ring-type: cobwebby=c,evanescent=e,flaring=f,large=l,none=n,pendant=p,sheathing=s,zone=z
* spore-print-color: black=k,brown=n,buff=b,chocolate=h,green=r,orange=o,purple=u,white=w,yellow=y
* population: abundant=a,clustered=c,numerous=n,scattered=s,several=v,solitary=y
* habitat: grasses=g,leaves=l,meadows=m,paths=p,urban=u,waste=w,woods=d

Let's load the dataset into so called Pandas dataframe.

In [15]:
mushrooms_df = pd.read_csv('mushrooms.csv')

Now we can take a closer look at the data.

In [16]:
mushrooms_df

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l


You can also print an overview of all attributes with the counts of unique values.

In [98]:
mushrooms_df.describe().T

,count,mean,std,min,25%,50%,75%,max
class,8124.0,0.482029,0.499708,0.0,0.0,0.0,1.0,1.0
cap-shape,8124.0,3.348104,1.604329,0.0,2.0,3.0,5.0,5.0
cap-surface,8124.0,1.827671,1.229873,0.0,0.0,2.0,3.0,3.0
cap-color,8124.0,4.504677,2.545821,0.0,3.0,4.0,8.0,9.0
bruises,8124.0,0.415559,0.492848,0.0,0.0,0.0,1.0,1.0
odor,8124.0,4.144756,2.103729,0.0,2.0,5.0,5.0,8.0
gill-attachment,8124.0,0.974151,0.158695,0.0,1.0,1.0,1.0,1.0
gill-spacing,8124.0,0.161497,0.368011,0.0,0.0,0.0,0.0,1.0
gill-size,8124.0,0.309207,0.462195,0.0,0.0,0.0,1.0,1.0
gill-color,8124.0,4.810684,3.540359,0.0,2.0,5.0,7.0,11.0


The dataset is pretty much balanced. That's a good news for the evaluation.

## Dataset Preprocessing

As our dataset consist of nominal/categorical values only, we will encode the strings into integers which will allow us to use similiraty measures such as Euclidean distance.

In [18]:
def encode_labels(df):
    import sklearn.preprocessing
    encoder = {}
    for col in df.columns:
        le = sklearn.preprocessing.LabelEncoder()
        le.fit(df[col])
        df[col] = le.transform(df[col])
        encoder[col] = le
    return df, encoder    

mushrooms_encoded_df, encoder = encode_labels(mushrooms_df)

In [11]:
mushrooms_encoded_df

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,0,3,2,4,0,5,0,0,0,11,...,2,5,5,0,1,1,4,0,1,2
8120,0,5,2,4,0,5,0,0,0,11,...,2,5,5,0,0,1,4,0,4,2
8121,0,2,2,4,0,5,0,0,0,5,...,2,5,5,0,1,1,4,0,1,2
8122,1,3,3,4,0,8,1,0,1,0,...,1,7,7,0,2,1,0,7,4,2


## Dataset Splitting

Before we start with the implementation of our k-nearest neighbors algorithm we need to prepare our dataset for the "training" and testing.

First, we divide the dataset into attributes (often called features) and classes (often called targets). Keeping attributes and classes separately is a common practice in many implementations. This should simplify the implementation and make the code understandable.

In [31]:
X_df = mushrooms_encoded_df.drop('class', axis=1)  # attributes
y_df = mushrooms_encoded_df['class']  # classes

X_array = X_df.to_numpy() 
y_array = y_df.to_numpy() 

# X_array = X_df.as_matrix()
# y_array = y_df.as_matrix()

And this is how it looks like.

In [87]:
print('X =', X_array)
print('len(X) =', len(X_array))
print('y =', y_array)
print('len(y) =', len(y_array))

X = [[5 2 4 ... 2 3 5]
 [5 2 9 ... 3 2 1]
 [0 2 8 ... 3 2 3]
 ...
 [2 2 4 ... 0 1 2]
 [3 3 4 ... 7 4 2]
 [5 2 4 ... 4 1 2]]
len(X) = 8124
y = [1 0 0 ... 0 1 0]
len(y) = 8124


Next, we need to split the attributes and classes into training sets and test sets.

**Exercise:**

Implement the holdout splitting method with shuffling.

In [103]:
def train_test_split(X, y, test_size=0.2):
    """
    Shuffles the dataset and splits it into training and test sets.
    
    :param X
        attributes
    :param y
        classes
    :param test_size
        float between 0.0 and 1.0 representing the proportion of the dataset to include in the test split
    :return
        train-test splits (X-train, X-test, y-train, y-test)
    """
    ### START CODE HERE ###
    from math import ceil
    if (test_size < 0 or test_size > 1):
        raise ValueError("Test size must be a value between 0 and 1")
    return X[int(ceil(test_size * len(X))):], y[0: int(ceil(test_size * len(y)))], X[int(ceil(test_size * len(X))):], y[0: int(ceil(test_size * len(y)))]
    # train_index, test_index = 0, 0
    # for i in range(len(X)):
    #     if (i > len(X) * test_size):
    #         X_train[train_index] = X[i]
    #         y_train[train_index] = y[i]
    #         train_index += 1
    #     else:
    #         X_test[test_index] = X[i]
    #         y_test[test_index] = y[i]
    #         test_index += 1
    # print(len(X_test), len(y_test))
    ### END CODE HERE ###

Let's split the dataset into training and validation/test set with 67:33 split.

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X_array, y_array, 0.33)

In [82]:
print('X_train =', X_train)
print('y_train =', y_train)
print('X_test =', X_test)
print('y_test =', y_test)

X_train = [6.91085226e-310 4.43813952e-316 0.00000000e+000 ... 4.00000000e+000
 1.00000000e+000 2.00000000e+000]
y_train = [6.91085226e-310 4.12893901e-316 4.22232691e-316 ... 4.00000000e+000
 1.00000000e+000 2.00000000e+000]
X_test = [4.1718089e-316 4.5615010e-316 4.1718089e-316 ... 0.0000000e+000
 0.0000000e+000 0.0000000e+000]
y_test = [4.04900235e-316 6.91085226e-310 4.04900235e-316 ... 0.00000000e+000
 0.00000000e+000 0.00000000e+000]


A quick sanity check...

In [105]:
assert len(X_train) == len(y_train)
assert len(y_train) == 5443
assert len(X_test) == len(y_test)
assert len(y_test) == 2681

## Algorithm

The k-nearest neighbors algorithm doesn't require a training step. The class of an unseen sample is deduced by comparison with samples of known class.

**Exercise:**

Implement the k-nearest neighbors algorithm.

In [106]:
# Use this section to place any "helper" code for the `knn()` function.

### START CODE HERE ###


def distance(a, b):
    return None

### END CODE HERE ###

In [114]:
def knn(X_true, y_true, X_pred, k=5):
    """
    k-nearest neighbors classifier.
    
    :param X_true
        attributes of the groung truth (training set)
    :param y_true
        classes of the groung truth (training set)
    :param X_pred
        attributes of samples to be classified
    :param k
        number of neighbors to use
    :return
        predicted classes
    """
    ### START CODE HERE ### 

    m = 100
    for i in range(m):
        print("something")
    
    y_pred = None
    ### END CODE HERE ### 
    return y_pred

In [116]:
y_hat = knn(X_train, y_train, X_test, k=5)

something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something
something


First ten predictions of the test set.

In [117]:
y_hat[:10]

TypeError: 'NoneType' object is not subscriptable

## Evaluation

Now we would like to assess how well our classifier performs.

**Exercise:**

Implement a function for calculating the accuracy of your predictions given the ground truth and predictions.

In [ ]:
def evaluate(y_true, y_pred):
    """
    Function calculating the accuracy of the model on the given data.
    
    :param y_true
        true classes
    :paaram y
        predicted classes
    :return
        accuracy
    """
    ### START CODE HERE ### 

    ### END CODE HERE ### 
    return accuracy

In [ ]:
accuracy = evaluate(y_test, y_hat)
print('accuracy =', accuracy)

How many items where misclassified?

In [ ]:
print('misclassified =', sum(abs(y_hat - y_test)))

How balanced is our test set?

In [ ]:
np.bincount(y_test)

If it's balanced, we don't have to be worried about objectivity of the accuracy metric.

---

Congratulations! At this point, hopefully, you have successufuly implemented a k-nearest neighbors algorithm able to classify unseen samples with high accuracy.

✌️